# Use DLite strategies from OTEAPI-OPTIMADE

This example shows how to use the DLite strategies from the OTEAPI-OPTIMADE plugin.

[DLite](https://github.com/SINTEF/dlite) is a Python library for working with data models and semantics.
It is considered to be the default semantic data model backend for use with OTEAPI.

To see more fundamental examples of how to use OTEAPI-OPTIMADE, see the example [_Use OTEAPI-OPTIMADE with OTElib_](/oteapi-optimade/examples/otelib/).
OTElib will also be used as a client in the current example.
Furthermore, only the HTTP requests-based backend will be used in this example.

## Setup

Please see the [setup instructions in _Use OTEAPI-OPTIMADE with OTElib_](/oteapi-optimade/examples/otelib/#setup) for how to ensure you have a proper environment to run the example in.

## Example

In this example, we will use the DLite strategies to query the [Materials Project](https://www.materialsproject.org/) OPTIMADE API.
We are interested in finding all structures that include the elements `Si` and `O`, and that have a maximum of 4 elements in total.

### Create a client

Let's start by initializing a client:

In [ ]:
from otelib import OTEClient

client = OTEClient("http://localhost:80")

### Data Resource strategy

The general pipeline is the same as it was for [_Use OTEAPI-OPTIMADE with OTElib_](otelib##search-through-materials-project-for-all-structures-with-the-formula-al2o3):

<!-- The mkdocs-jupyter plugin creates a folder 'otelib' wherein it puts the generated HTML
Therefore, we here use the "full" path (minus the base URL) to reference the image -->
![Pipeline example 1](/oteapi-optimade/img/chem_formula_pipeline.png)

However, in order to use DLite we need to _either_ reference a specific `accessService` **or** set a flag in the configuration of the data resource strategy.

Note, it is _only_ for the data resource strategy we need to specify the usage of DLite, as the filter strategy is generic and merely a helper for more explicitly setting the query parameters to be used for the underlying OPTIMADE query.

In [ ]:
data_resource_strategy = client.create_dataresource(
    accessService="OPTIMADE+DLite",
    accessUrl="https://optimade.materialsproject.org",
)

# This is equivalent to:
# data_resource_strategy = client.create_dataresource(
#     accessService="OPTIMADE",
#     accessUrl="https://optimade.materialsproject.org",
#     configuration={"use_dlite": True},
# )

### Filter strategy

The OPTIMADE filter query to fulfill [the interests outlined above](#example) should look like this:

```
elements HAS ALL "Si","O" AND nelements<=4
```

In [ ]:
filter_strategy = client.create_filter(
    filterType="OPTIMADE",
    query='elements HAS ALL "Si","O" AND nelements<=4',
)

#### Setup, execute and inspect the pipeline

In [ ]:
import json

pipeline = filter_strategy >> data_resource_strategy
session = pipeline.get()
# parsed_session = json.loads(session)
# parsed_session.keys()